In [1]:
import dspy

from web_opt.server.web_opt_environment import WebOptEnvironment
from web_opt.models import WebOptAction, WebOptObservation, WebOptState, WebsiteState, LighthouseScores, VerificationScores
from web_opt import WebOptAction, WebOptEnv

In [ ]:
lm = dspy.LM(
    "anthropic/claude-sonnet-4-5-20250929",
    api_key="" # Add your Anthropic API key here
)
dspy.configure(lm=lm)

In [3]:
class WebAgentSignature(dspy.Signature):
    """Modify a single file in the codebase to imporve website performance, SEO, and accessibility."""
    code: dict[str, str] = dspy.InputField(desc="A dictionary mapping file paths to their corresponding code content.")
    file_name: str = dspy.OutputField(desc="The file path of the file to be modified.")
    modified_code: str = dspy.OutputField(desc="The modified code of the specified file.")

# web_agent = dspy.Predict(WebAgentSignature)
web_agent = dspy.ChainOfThought(WebAgentSignature)

In [4]:
env = WebOptEnv(base_url="http://localhost:8000")
obs = env.reset()
print(f'Initial Reward: {obs.reward}')

for i in range(5):
    prediction = web_agent(code=obs.observation.site.code)
    step_code = {
        prediction.file_name: prediction.modified_code
    }
    print(f"Modifying file: {prediction.file_name}")
    action = WebOptAction(site=WebsiteState(code=step_code))
    obs = env.step(action)
    print(f"Reward: {obs.reward}")
    print(f"Step {i+1} completed.\n{'='*60}\n")

Initial Reward: 0
Modifying file: /tmp/current_project/src/pages/Home.jsx
Reward: 0.6112568039520013
Step 1 completed.

Modifying file: /tmp/current_project/src/pages/Home.css
Reward: 0.0
Step 2 completed.

Modifying file: /tmp/current_project/src/pages/Home.jsx
Reward: -0.6276425293325595
Step 3 completed.

Modifying file: /tmp/current_project/src/pages/Home.css
Reward: 0.0
Step 4 completed.

Modifying file: /tmp/current_project/src/main.jsx
Reward: 0.2902594403503856
Step 5 completed.

